# Clean Paypal and identify plan duration using CM data

###Specifications

####  Paypal:

We need to clean the Paypal statement to move the Non-Revenue transactions.
Create a separate sheet and move all of the below data sets

Hold on Available Balance
Reversal of General Account Hold
Hold on Balance for Dispute Investigation
Cancellation of Hold for Dispute Resolution
General Payment
General Withdrawal
PreApproved Payment Bill User Payment
Dispute Fee
Chargeback
Account Hold for Open Authorization
Payment Reversal
Chargeback Reversal
Fee Reversal
Mass Pay Payment
Mass Pay Reversal

This will leave us with the below categories; Further, clean up needed here

General Currency Conversion -------> Move all EUR to the above sheet
Subscription Payment  -----> Move all Negative values from this category only to the above
Payment Refund - Keep Data

Final Step -----> Now we need to compare this clean-up data ( Revenue and Refunds ) with the CM Paypal created through to identify if the charges are annual or Monthly.



V2:
GAAP Revenue Recognition -The purpose of doing this is to get help with monthly GAAP revenue recognition.

o To the paypal cleaned file add a new column to the "Cleaned Data" Tab, then recognize data either by Monthly or Annual based on the CM Paypal data. Sort any data you cannot recognize as either Mon or Annual to the bottom.

o Stripe- Add a new column to Stripe data and then perform the above check with CM Stripe data to identify each item by Mon or Annual. There are few non-revenue Stripe data like fees /disputes. Move these to the bottom before doing this.


### Script
---

#### Imports,prepartions and functions

In [1]:
import pandas as pd
import os
!pip install XlsxWriter
import xlsxwriter


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 KB 9.1 MB/s eta 0:00:00


###### Preparations
---


In [10]:
#Mount GDrive
from google.colab import drive
drive.mount('/content/gdrive')

#Set file paths
path = "/content/gdrive/My Drive/Data Science 2022/CM Audit/"

filepp = path + "Paypal Data/2023/Paypal_Raw_Data_2023_02.xlsx"
filecm  =path + "Monthly Output/2023.02/CM_2023.02.xlsx"

outfile = path + "Monthly Output/2023.02/PayPal_2023.02.xlsx"


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


#### Cleanup Paypal and  Compare with CM data to identify if the charges are annual or monthly

In [12]:
lstType = ['Hold on Available Balance','Reversal of General Account Hold',
          'Hold on Balance for Dispute Investigation','Cancellation of Hold for Dispute Resolution',
          'General Payment','General Withdrawal','PreApproved Payment Bill User Payment',
          'Dispute Fee','Chargeback','Account Hold for Open Authorization','Payment Reversal','Chargeback Reversal',
          'Fee Reversal','Mass Pay Payment','Mass Pay Reversal']


#For csv format
if filepp[-3:] == "csv":  
  df = pd.read_csv(filepp , na_filter=False, index_col=False)
else:
  df = pd.read_excel(filepp , na_filter=False, index_col=False)

#Change the data type of gross to numeric
df = df.astype({"Gross": str})
df['Gross'] = df['Gross'].str.replace(',','')
df = df.astype({"Gross": float})


############ Segregate Non-Revenue transactions based on 'Type'
#Move the below 'Types' to seperate sheet - Non revenue
#Hold on Available Balance,Reversal of General Account Hold,Hold on Balance for Dispute Investigation,Cancellation of Hold for Dispute Resolution,General Payment,General Withdrawal,
#PreApproved Payment Bill User Payment,Dispute Fee,Chargeback,Account Hold for Open Authorization,Payment Reversal,Chargeback Reversal,Fee Reversal,Mass Pay Payment,Mass Pay Reversal
dfmove = df[df['Type'].isin(lstType)] #dataset to be moved to 'others' sheet
dfclean = df[~df['Type'].isin(lstType)] #clean dataset


############# Filter 'General currency conversion' with 'EUR'
#dfclean1 = dfclean[~((dfclean['Type'] == "General Currency Conversion") & (dfclean['Currency'] != "USD"))]
#dfeuro = dfclean[((dfclean['Type'] == "General Currency Conversion") & (dfclean['Currency'] != "USD"))]
dfclean1 = dfclean[~(dfclean['Currency'] != "USD")]
dfeuro = dfclean[(dfclean['Currency'] != "USD")]
dfmove1= dfmove.append(dfeuro)

############# Filter negative values under 'Subscription payment'
dfclean2 = dfclean1[~((dfclean1['Type'] == "Subscription Payment") & (dfclean1['Gross'] < 0))]
dfmove2 = dfmove1.append(dfclean1[((dfclean1['Type'] == "Subscription Payment") & (dfclean1['Gross'] < 0))])

############# **** Identify plan type using CM data ** #####################
dfpp = dfclean2.copy()
dfpp.rename(columns={"Reference Txn ID": 'Customer External ID'}, inplace = True)
#Load CM file
dfcm = pd.read_excel(filecm,sheet_name="Paypal" , na_filter=False, index_col=False)

dfmon = pd.merge(dfpp, dfcm, on="Customer External ID", how='left')

#Keep only paypal data along with plan
cmcols = dfcm.columns.tolist()
cmcols.remove('Plan Type')
cmcols.remove('Customer External ID')
mcols = dfmon.columns.tolist()
newcols = [x for x  in mcols if x not in cmcols  ]
dfnewpp = dfmon[newcols]

#Rename paypal column back to 'Reference Txn ID' 
dfnewpp.rename(columns={'Customer External ID': 'Reference Txn ID'}, inplace = True)

#Delete duplicate rows
dfnewpp = dfnewpp.drop_duplicates()

############## Fill out plan type for 'general currency conversion' #################
#1. Pick 'General currency conversion' records in cleaned dataset
#2. Map "Reference Txn ID" with the 'Transaction ID' of 'non revenue' records and find 
#   corresponding 'Ref Txn ID' . This id will start with 'I-'
#3. Map this new "Reference Txn ID" with CM data to get the plan
#####################################################################################
dfcurr = dfnewpp[dfnewpp['Type'] == "General Currency Conversion"]    #Pick 'General currency conversion' records in cleaned dataset
indall=[]
curr_dic = dfcurr.to_dict('records')    #convert to dictionary for iteration efficiency


for row in curr_dic:
  item = row['Reference Txn ID']            
  dftemp = dfmove2[dfmove2['Transaction ID'] == item]   #Map "Reference Txn ID" with the 'Transaction ID' of 'non revenue' records
  
  inds = dfmove2[dfmove2['Transaction ID'] == item].index   #Drop the corresponding records in non revenue records
  
  indall.append(inds[0])
  

  id = dftemp['Reference Txn ID'].tolist()[0]           #find corresponding 'Ref Txn ID' . This id will start with 'I-'
  plan = dfcm[dfcm['Customer External ID'] == id]['Plan Type'] 
  
  if not plan.empty:                                    #Subsciption refund will have empty values
    row['Plan Type'] = plan.values[0]

dfcurr = pd.DataFrame.from_dict(curr_dic)


dfmove2.drop (indall,inplace=True)        #Drop all curr records in non revenue dataset
#Append new currency conversion records 
dfnewpp = pd.concat([dfnewpp,dfcurr])

'''
##############################################################################################################################################
#few records that are without plan whose 'Reference transaction Id' is mapping with part of strings under 'Charge Id', Invoice Id'  and 'Line Item Name' of CM data. 
#Map those and fill out the plan type
##############################################################################################################################################
#Get records that are without plan type
#Map 'ref txn id' of these records with the CM's ChatgeId column to find if the string is contained in it. If so, pick the plan
dfempty = dfnewpp[dfnewpp['Plan Type'].isnull()]
dfnewpp = dfnewpp[~dfnewpp['Plan Type'].isnull()]

dict_emp = dfempty.to_dict('records')      
for row in dict_emp:
  x = row['Reference Txn ID']
  dfchargeid=pd.DataFrame()
  dfchargeid = dfcm[dfcm['Charge ID'].str.contains(str(x))]   #If charge id contains ref txn id then pick the plan type from that row
  if not dfchargeid.empty:
    dfchargeid.reset_index(drop=True, inplace=True)
    row['Plan Type'] = dfchargeid.iloc[0]['Plan Type']
dffilled = pd.DataFrame.from_dict(dict_emp)

dfnewpp = pd.concat([dfnewpp,dffilled])
dfnewpp.drop_duplicates(inplace=True)
#Sort by plan type
dfnewpp.sort_values('Plan Type',ascending=False,inplace=True)
'''

writer = pd.ExcelWriter(outfile, engine='xlsxwriter')
dfnewpp.to_excel(writer,sheet_name="Cleaned Data", index=False)
dfmove2.to_excel(writer,sheet_name="Non Revenue records", index=False)

writer.save()
writer.close()
    

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/usr/local/lib/python3.8/dist-packages/xlsxwriter/workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
